In [25]:
import tkinter as tk
from tkinter import filedialog
import cv2
from PIL import Image, ImageTk

class ImageProcessingGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Processing GUI")
        self.original_img = None

        # Label to display the heading "Choose an Image"
        self.label_heading = tk.Label(root, text="Choose an Image", font=("Helvetica", 18, "bold"))
        self.label_heading.pack(pady=10)

        # Button to open the file dialog for image selection
        self.btn_open_image = tk.Button(root, text="Open Image", command=self.open_image)
        self.btn_open_image.pack(pady=5)

        # Label to display the selected image
        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

    def open_image(self):
        # Open file dialog to select an image file
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            # Read the image using OpenCV and display it on the image_label
            self.original_img = cv2.imread(file_path)
            img = Image.open(file_path)
            img = img.resize((400, 300))
            img = ImageTk.PhotoImage(img)
            self.image_label.config(image=img)
            self.image_label.image = img

            # Process image
            self.process_image()

    def process_image(self):
        if self.original_img is not None:
            processed_img = self.original_img.copy()

            # Perform image processing operations
            gray_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
            blurred_img = cv2.GaussianBlur(processed_img, (5, 5), 0)
            edges_img = cv2.Canny(processed_img, 100, 200)

            # Check OpenCV version for findContours function
            contours_version = cv2.__version__.split('.')[0]
            if int(contours_version) >= 4:
                contours, _ = cv2.findContours(edges_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            else:
                _, contours, _ = cv2.findContours(edges_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw contours on the processed image
            cv2.drawContours(processed_img, contours, -1, (0, 255, 0), 2)

            # Display processed images
            self.show_processed_images(gray_img, blurred_img, edges_img, processed_img)

    def show_processed_images(self, gray_img, blurred_img, edges_img, contour_img):
        # Resize images to medium size for display
        gray_img = cv2.resize(gray_img, (400, 300))
        blurred_img = cv2.resize(blurred_img, (400, 300))
        edges_img = cv2.resize(edges_img, (400, 300))
        contour_img = cv2.resize(contour_img, (400, 300))

        # Convert images to RGB for display with tkinter
        gray_img = cv2.cvtColor(gray_img, cv2.COLOR_BGR2RGB)
        blurred_img = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2RGB)
        edges_img = cv2.cvtColor(edges_img, cv2.COLOR_BGR2RGB)
        contour_img = cv2.cvtColor(contour_img, cv2.COLOR_BGR2RGB)

        # Convert images to PIL format
        gray_img = Image.fromarray(gray_img)
        blurred_img = Image.fromarray(blurred_img)
        edges_img = Image.fromarray(edges_img)
        contour_img = Image.fromarray(contour_img)

        # Display images with respective headings
        self.show_image_with_heading("Gray Image", gray_img)
        self.show_image_with_heading("Blurred Image", blurred_img)
        self.show_image_with_heading("Edges Image", edges_img)
        self.show_image_with_heading("Contours Image", contour_img)

    def show_image_with_heading(self, heading, image):
        # Resize image to fit in the window
        image = image.resize((400, 300))
        image = ImageTk.PhotoImage(image)

        # Create a new window to display the image with its heading
        window = tk.Toplevel()
        window.title(heading)
        label = tk.Label(window, image=image)
        label.image = image
        label.pack(pady=10)

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageProcessingGUI(root)
    root.mainloop()
